In [ ]:
import requests
from bs4 import BeautifulSoup as be

url_prefix_zh = "https://ys.mihoyo.com/content/ysCn/getContentList?"

url_mondstadt_zh = url_prefix_zh + "pageSize=20&pageNum=1&order=asc&channelId=150"
url_liyue_zh = url_prefix_zh + "pageSize=20&pageNum=1&order=asc&channelId=151"
url_inazuma_zh = url_prefix_zh + "pageSize=20&pageNum=1&order=asc&channelId=324"
url_sumeru_zh = url_prefix_zh + "pageSize=20&pageNum=1&order=asc&&channelId=488"
url_fontaine_zh = url_prefix_zh + "pageSize=20&pageNum=1&order=asc&channelId=324"
url_natlan_zh = url_prefix_zh + "pageSize=20&pageNum=1&order=asc&channelId=324"
url_snezhnaya_zh = url_prefix_zh + "pageSize=20&pageNum=1&order=asc&channelId=324"

url_prefix_en = "https://content-static-sea.hoyoverse.com/content/yuanshen/getContentList?"

url_mondstadt_en = url_prefix_en + "pageSize=4&pageNum=1&channelId=96"
url_liyue_en = url_prefix_en + "channelId=874&pageSize=1&pageNum=1&order=asc"
url_inazuma_en = url_prefix_en + "channelId=775&pageSize=1&pageNum=1&order=asc"
url_sumeru_en = url_prefix_en + "channelId=874&pageSize=1&pageNum=1&order=asc"
url_fontaine_en = url_prefix_en + "pageSize=20&pageNum=1&order=asc&channelId=324"
url_natlan_en = url_prefix_en + "pageSize=20&pageNum=1&order=asc&channelId=324"
url_snezhnaya_en = url_prefix_en + "pageSize=20&pageNum=1&order=asc&channelId=324"

zh_url = [url_mondstadt_zh, url_liyue_zh, url_inazuma_zh, url_sumeru_zh]
en_url = [url_mondstadt_en, url_liyue_en, url_inazuma_en, url_sumeru_en]


def get_json(_url_):
    req = requests.get(url=_url_)
    if req.status_code == 200:
        return req.json()['data']
    else:
        return None


def clean_data(_data_):
    _return_ = []
    for key in _data_['list']:
        ext = key["ext"]
        all_data = {key['title']: {
            "角色ICON": ext[0]["value"][0]["url"],
            "电脑端立绘": ext[1]["value"][0]["url"],
            "手机端立绘": ext[15]["value"][0]["url"],
            "角色名字": key['title'],
            "角色属性": ext[3]["value"][0]["url"],
            "角色语言": ext[4]["value"],
            "声优1": ext[5]["value"],
            "声优2": ext[6]["value"],
            "简介": be(ext[7]["value"], "lxml").p.text.strip(),
            "台词": ext[8]["value"][0]["url"],
            "音频": {
                ext[9]["value"][0]["name"]: ext[9]["value"][0]["url"],
                ext[10]["value"][0]["name"]: ext[10]["value"][0]["url"],
                ext[11]["value"][0]["name"]: ext[11]["value"][0]["url"],
                ext[12]["value"][0]["name"]: ext[12]["value"][0]["url"],
                ext[13]["value"][0]["name"]: ext[13]["value"][0]["url"],
                ext[14]["value"][0]["name"]: ext[14]["value"][0]["url"],
            },
        }
        }
        _return_.append(all_data[key['title']])
    return _return_


def data(lang):
    _json_ = {}
    for url in lang:
        jsonlist = clean_data(get_json(url))
        for json in jsonlist:
            _json_[json['角色名字']] = json
    return _json_


def lookup(name):
    json = data(zh_url)[name]
    print("查找角色：", name)
    for key, value in json.items():
        if key == "音频":
            for keys, values in json[key].items():
                print(f"{keys}：{values}")
        else:
            print(f"{key}：{value}")


if __name__ == '__main__':
    data(zh_url)
    data(en_url)
